In [28]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from PIL import Image
import os
from pathlib import Path
from utils import *
from allResNets import *
from engine import *
from data_transforms import create_transform
from dataloaders import *

change_to_disk()
data_dir = Path("clean_split_1k/")
models_path = Path('Quality/Models_test/')
check_dir(models_path)

In [29]:
image_shape = (1080, 1920)
resize_factor = 0.3
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]
BATCH_SIZE = 16

data_transform = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=True, gauss_noise=True)

train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders_multilabel(data_dir, transform = data_transform, batch_size=BATCH_SIZE, num_workers= NUM_WORKERS)

In [30]:
NUM_EPOCHS = 20
lr = 0.001
device = get_device()

resnet18 = ResNet18()

#Combines the sigmoid with the BinaryCrossEntropy
loss_fn = nn.BCEWithLogitsLoss()

In [31]:
model_name = "ResNet18"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet18
resnet18.to(device)
nadam_optim = torch.optim.NAdam(params=resnet18.parameters(), lr=lr)
train_resnet18_results, train_time_resnet18 = train(resnet18, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.3796 | train_acc: 0.2655 | validation_loss: 0.6396 | validation_acc: 0.3105
Epoch: 2 | train_loss: 0.3080 | train_acc: 0.3917 | validation_loss: 1.7826 | validation_acc: 0.1992
Epoch: 3 | train_loss: 0.2798 | train_acc: 0.4640 | validation_loss: 0.2736 | validation_acc: 0.4492
Epoch: 4 | train_loss: 0.2615 | train_acc: 0.5110 | validation_loss: 1.0050 | validation_acc: 0.2969
Epoch: 5 | train_loss: 0.2392 | train_acc: 0.5670 | validation_loss: 0.8186 | validation_acc: 0.2812
Epoch: 6 | train_loss: 0.2146 | train_acc: 0.6232 | validation_loss: 1.0033 | validation_acc: 0.4141
Epoch: 7 | train_loss: 0.1873 | train_acc: 0.6860 | validation_loss: 1.0143 | validation_acc: 0.3008
Epoch: 8 | train_loss: 0.1896 | train_acc: 0.6833 | validation_loss: 2.8340 | validation_acc: 0.2539
Epoch: 9 | train_loss: 0.1696 | train_acc: 0.7210 | validation_loss: 0.7490 | validation_acc: 0.4844
Epoch: 10 | train_loss: 0.1614 | train_acc: 0.7352 | validation_loss: 0.3747 | validation_a

In [20]:
loaded_model = load_model(ResNet18, Path("Quality/Models_test/ResNet18/ResNet18_2_2_epcs.pth"),device)

In [21]:
transforms = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=True, gauss_noise=True)

In [22]:
img = Image.open("clean_split_1k/test/2_Good/Vis_good_1_0_frame300.png")
img = transforms(img)

In [23]:
loaded_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (identity_downsample): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [24]:
y = loaded_model(img.unsqueeze(0))
y = torch.sigmoid(y)
y = torch.round(y)
y

tensor([[0., 0., 1., 0., 0.]], grad_fn=<RoundBackward0>)